In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms

import numpy as np
from tqdm import tqdm

# Check and set the device (CPU, CUDA, or MPS)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# -----------------------------
# 1. Data Preparation
# -----------------------------

# Define transformations: downsample to 16x16 and convert images to tensors
transform = transforms.Compose([
    transforms.Resize((16, 16)),  # Downsample images for faster computation
    transforms.ToTensor(),        # Convert PIL images to tensors
])

# Load CIFAR-10 training and test datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)

# Define DataLoaders with num_workers=0 to avoid multiprocessing issues
batch_size = 128  # Batch size for training the autoencoder

train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=0)  # Set num_workers=0
test_loader = DataLoader(test_dataset, batch_size=batch_size,
                         shuffle=False, num_workers=0)  # Set num_workers=0

# -----------------------------
# 2. Define Autoencoder Architecture
# -----------------------------

class Autoencoder(nn.Module):
    def __init__(self, latent_dim=128):
        super(Autoencoder, self).__init__()
        # Encoder: 3 Convolutional layers
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),  # Output: 32 x 8 x 8
            nn.ReLU(True),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1), # Output: 64 x 4 x 4
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),# Output: 128 x 2 x 2
            nn.ReLU(True),
            nn.Flatten(),                                         # Flatten to 512 (128*2*2)
            nn.Linear(128*2*2, latent_dim)                        # Latent vector
        )
        # Decoder: 3 Transposed Convolutional layers
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128*2*2),                       # Expand latent vector
            nn.ReLU(True),
            nn.Unflatten(1, (128, 2, 2)),                         # Reshape to 128 x 2 x 2
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1), # 4x4
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  # 8x8
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 3, kernel_size=3, stride=2, padding=1, output_padding=1),   # 16x16
            nn.Sigmoid()                                           # Output values between 0 and 1
        )
    
    def forward(self, x):
        latent = self.encoder(x)  # Encode input to latent space
        reconstructed = self.decoder(latent)  # Decode latent vector to reconstruct input
        return reconstructed

# Initialize the autoencoder and move it to the appropriate device
autoencoder = Autoencoder(latent_dim=128).to(device)

# -----------------------------
# 3. Feature Extraction Function
# -----------------------------

def extract_features(model, dataloader):
    """
    Extract latent features from the dataset using the encoder part of the autoencoder.

    Args:
        model (nn.Module): The autoencoder model.
        dataloader (DataLoader): DataLoader for the dataset.

    Returns:
        features (torch.Tensor): Extracted latent features.
        labels (torch.Tensor): Corresponding labels.
    """
    model.eval()  # Set model to evaluation mode
    features = []
    labels = []
    with torch.no_grad():  # Disable gradient computation
        for data, target in tqdm(dataloader, desc="Extracting Features"):
            data = data.to(device)  # Move data to device
            latent = model.encoder(data)  # Get latent features
            features.append(latent.cpu())  # Move features to CPU and store
            labels.append(target)  # Store labels
    features = torch.cat(features, dim=0)  # Concatenate all features
    labels = torch.cat(labels, dim=0)      # Concatenate all labels
    return features, labels

# -----------------------------
# 4. Extract Initial Features (H0) with Random Autoencoder
# -----------------------------

print("Extracting H0 (Random Projection) Features...")
H0_train, y0_train = extract_features(autoencoder, train_loader)
H0_test, y0_test = extract_features(autoencoder, test_loader)

print(f'H0_train shape: {H0_train.shape}')
print(f'H0_test shape: {H0_test.shape}')

# -----------------------------
# 5. Train the Autoencoder
# -----------------------------

# Define the loss function (Mean Squared Error) and the optimizer (Adam)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3, weight_decay=1e-5)

num_epochs = 20  # Number of epochs for training

print("\nStarting Autoencoder Training...")
for epoch in range(num_epochs):
    autoencoder.train()  # Set model to training mode
    running_loss = 0.0
    loop = tqdm(train_loader, desc=f'Epoch [{epoch+1}/{num_epochs}]')
    for data, _ in loop:
        data = data.to(device)  # Move data to device

        # Forward pass: reconstruct the input
        reconstructed = autoencoder(data)
        loss = criterion(reconstructed, data)  # Compute reconstruction loss

        # Backward pass and optimization
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()        # Backpropagate the loss
        optimizer.step()       # Update the weights

        running_loss += loss.item() * data.size(0)  # Accumulate loss
        loop.set_postfix(loss=loss.item())         # Update progress bar

    # Compute average loss for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

print('Finished Training Autoencoder')

# -----------------------------
# 6. Extract Trained Features (H1) with Trained Autoencoder
# -----------------------------

print("\nExtracting H1 (Trained Features) Features...")
H1_train, y1_train = extract_features(autoencoder, train_loader)
H1_test, y1_test = extract_features(autoencoder, test_loader)

print(f'H1_train shape: {H1_train.shape}')
print(f'H1_test shape: {H1_test.shape}')

# -----------------------------
# 7. Define and Train Classifiers
# -----------------------------

class Classifier(nn.Module):
    def __init__(self, input_dim=128, num_classes=10):
        super(Classifier, self).__init__()
        # Simple feedforward network with one hidden layer
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),  # Input layer
            nn.ReLU(True),             # Activation
            nn.Linear(64, num_classes) # Output layer
        )
    
    def forward(self, x):
        out = self.fc(x)  # Forward pass
        return out

class FeatureDataset(Dataset):
    """
    Custom Dataset for handling pre-extracted features and labels.
    """
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create datasets for H0 (Random Projection)
H0_train_dataset = FeatureDataset(H0_train, y0_train)
H0_test_dataset = FeatureDataset(H0_test, y0_test)

# Create datasets for H1 (Trained Features)
H1_train_dataset = FeatureDataset(H1_train, y1_train)
H1_test_dataset = FeatureDataset(H1_test, y1_test)

# Define DataLoaders for classifiers with num_workers=0
classifier_batch_size = 256  # Batch size for classifier training

H0_train_loader = DataLoader(H0_train_dataset, batch_size=classifier_batch_size,
                             shuffle=True, num_workers=0)  # Set num_workers=0
H0_test_loader = DataLoader(H0_test_dataset, batch_size=classifier_batch_size,
                            shuffle=False, num_workers=0)  # Set num_workers=0

H1_train_loader = DataLoader(H1_train_dataset, batch_size=classifier_batch_size,
                             shuffle=True, num_workers=0)  # Set num_workers=0
H1_test_loader = DataLoader(H1_test_dataset, batch_size=classifier_batch_size,
                            shuffle=False, num_workers=0)  # Set num_workers=0

def train_classifier(model, train_loader, test_loader, num_epochs=20, learning_rate=1e-3):
    """
    Train a classifier model and evaluate its accuracy on the test set.

    Args:
        model (nn.Module): The classifier model to train.
        train_loader (DataLoader): DataLoader for training data.
        test_loader (DataLoader): DataLoader for testing data.
        num_epochs (int): Number of training epochs.
        learning_rate (float): Learning rate for the optimizer.

    Returns:
        accuracy (float): Test accuracy of the trained classifier.
    """
    model = model.to(device)  # Move model to device
    criterion = nn.CrossEntropyLoss()  # Loss function for classification
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Optimizer

    print(f"\nStarting Training for {num_epochs} epochs...")
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        loop = tqdm(train_loader, desc=f'Classifier Epoch [{epoch+1}/{num_epochs}]')
        for features, labels in loop:
            features = features.to(device)  # Move features to device
            labels = labels.to(device)      # Move labels to device

            # Forward pass: compute predictions
            outputs = model(features)
            loss = criterion(outputs, labels)  # Compute loss

            # Backward pass and optimization
            optimizer.zero_grad()  # Zero the gradients
            loss.backward()        # Backpropagate the loss
            optimizer.step()       # Update the weights

            running_loss += loss.item() * features.size(0)  # Accumulate loss
            loop.set_postfix(loss=loss.item())             # Update progress bar

        # Compute average loss for the epoch
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    # Evaluation on the test set
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient computation
        for features, labels in tqdm(test_loader, desc="Evaluating"):
            features = features.to(device)  # Move features to device
            labels = labels.to(device)      # Move labels to device
            outputs = model(features)       # Get model predictions
            _, predicted = torch.max(outputs.data, 1)  # Get predicted classes
            total += labels.size(0)          # Total number of samples
            correct += (predicted == labels).sum().item()  # Correct predictions

    accuracy = 100 * correct / total  # Calculate accuracy
    return accuracy

# Initialize classifiers for H0 and H1
classifier_H0 = Classifier(input_dim=128, num_classes=10)
classifier_H1 = Classifier(input_dim=128, num_classes=10)

# -----------------------------
# 8. Train and Evaluate Classifier on H0 (Random Projection)
# -----------------------------

print("\nTraining classifier on H0 (Random Projection)")
accuracy_H0 = train_classifier(classifier_H0, H0_train_loader, H0_test_loader, num_epochs=20)
print(f'Classifier Accuracy on H0: {accuracy_H0:.2f}%')

# -----------------------------
# 9. Train and Evaluate Classifier on H1 (Trained Features)
# -----------------------------

print("\nTraining classifier on H1 (Trained Features)")
accuracy_H1 = train_classifier(classifier_H1, H1_train_loader, H1_test_loader, num_epochs=20)
print(f'Classifier Accuracy on H1: {accuracy_H1:.2f}%')

# -----------------------------
# 10. Compare Classifier Performance
# -----------------------------

print("\n--- Comparison of Classifier Performance ---")
print(f'Accuracy with H0 (Random Projection): {accuracy_H0:.2f}%')
print(f'Accuracy with H1 (Trained Features): {accuracy_H1:.2f}%')


Using device: mps
Files already downloaded and verified
Files already downloaded and verified
Extracting H0 (Random Projection) Features...


Extracting Features: 100%|██████████| 79/79 [00:00<00:00, 150.18it/s]


H0_train shape: torch.Size([50000, 128])
H0_test shape: torch.Size([10000, 128])

Starting Autoencoder Training...


Epoch [1/20]: 100%|██████████| 391/391 [00:05<00:00, 72.29it/s, loss=0.0144]


Epoch [1/20], Loss: 0.0233


...

Epoch [19/20], Loss: 1.3442


Classifier Epoch [20/20]: 100%|██████████| 196/196 [00:00<00:00, 307.99it/s, loss=1.25]


Epoch [20/20], Loss: 1.3394


Evaluating: 100%|██████████| 40/40 [00:00<00:00, 812.00it/s]

Classifier Accuracy on H1: 50.15%

--- Comparison of Classifier Performance ---
Accuracy with H0 (Random Projection): 29.44%
Accuracy with H1 (Trained Features): 50.15%
